In [3]:
import json
import random
import numpy as np
import nltk
import pickle
from nltk.stem import WordNetLemmatizer
lemmatizer=WordNetLemmatizer()
import tensorflow as tf
import keras
from keras.models import Sequential
from keras.layers import Dense,Activation,Dropout
from keras.optimizers import SGD
import spacy
from spacy.lang.en.stop_words import STOP_WORDS

In [7]:
# STOP_WORDS

In [4]:
intents=json.loads(open('online.json').read())

In [5]:
words=[]
classes=[]
documents=[]
ignore_letters=['?','!','.',',']

In [20]:
for intent in intents['online']:
    # print(intent['tag'])
    for pattern in intent['patterns']:
        word_list=nltk.word_tokenize(pattern)
        words.extend(word_list)
        documents.append((word_list,intent['tag']))
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

In [21]:
def get_response(intents_list,intents_json):
    tag=intents_list[0]['intent']
    list_of_intents=intents_json['intents']
    for i in list_of_intents:
        if i['tag']==tag:
            result=random.choice(i['responses'])
            break
    return result

In [22]:
words=[lemmatizer.lemmatize(word) for word in words if word not in ignore_letters]
words=sorted(set(words))
classes=sorted(set(classes))

In [23]:
pickle.dump(words,open('words.pkl','wb'))
pickle.dump(classes,open('classes.pkl','wb'))

In [24]:
training=[]
ouput_empty=[0]*len(classes)

In [25]:
for document in documents:
    bag=[]
    words_patterns=document[0]
    words_patterns=[lemmatizer.lemmatize(word.lower()) for word in words_patterns]
    for word in words:
        bag.append(1) if word in words_patterns else bag.append(0)
    output_row=list(ouput_empty)
    output_row[classes.index(document[1])] = 1
    training.append([bag,output_row])

In [26]:
random.shuffle(training)
training=np.array(training)

/tmp/ipykernel_35938/4111924452.py:2: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  training=np.array(training)


In [27]:
train_x=list(training[:,0])
train_y=list(training[:,1])

In [28]:
model=Sequential()
model.add(Dense(128,input_shape=(len(train_x[0]),),activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(64,activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(len(train_y[0]),activation='softmax'))

In [29]:
sgd=SGD(lr=0.01,decay=1e-6,momentum=0.9)
model.compile(loss='categorical_crossentropy',optimizer=sgd,metrics=['accuracy'])

In [30]:
hist=model.fit(np.array(train_x),np.array(train_y),epochs=200,batch_size=1,verbose=1)

Epoch 1/200
12/12 [==============================] - 0s 911us/step - loss: 1.1532 - accuracy: 0.2500
Epoch 2/200
12/12 [==============================] - 0s 948us/step - loss: 0.9768 - accuracy: 0.7500
Epoch 3/200
12/12 [==============================] - 0s 1ms/step - loss: 0.8478 - accuracy: 0.5000
Epoch 4/200
12/12 [==============================] - 0s 1ms/step - loss: 0.6191 - accuracy: 1.0000
Epoch 5/200
12/12 [==============================] - 0s 1ms/step - loss: 0.3149 - accuracy: 1.0000
Epoch 6/200
12/12 [==============================] - 0s 1ms/step - loss: 0.3428 - accuracy: 0.9167
Epoch 7/200
12/12 [==============================] - 0s 1ms/step - loss: 0.1137 - accuracy: 1.0000
Epoch 8/200
12/12 [==============================] - 0s 1ms/step - loss: 0.0596 - accuracy: 1.0000
Epoch 9/200
12/12 [==============================] - 0s 1ms/step - loss: 0.1059 - accuracy: 1.0000
Epoch 10/200
12/12 [==============================] - 0s 1ms/step - loss: 0.0786 - accuracy: 1.0000
Epoch

In [31]:
model.save('online.h5',hist)